In [ ]:
cd ..

# FileUtil Initialization

In [ ]:
from src.utils.file_util import FileUtil

file_util = FileUtil()

# Retrieve Raw Train Data

In [ ]:
train = file_util.get_raw_train_data()

In [ ]:
train.head()

# Preprocessing

In [ ]:
from src.preprocessing.transformations import apply_cleaning_train

In [ ]:
train = apply_cleaning_train(train)

In [ ]:
train.head()

# Training

## Sentiment Analysis

In [ ]:
from src.models.sentiment_analysis.train.train import sentiment_analysis_train

In [ ]:
sentiment_analysis_train()

## Topic Modelling

In [ ]:
from src.models.topic_modelling.train.train import topic_modelling_train

In [ ]:
topic_modelling_train()

# Evaluation

## Sentiment Analysis

In [ ]:
metrics = file_util.get_metrics("sentiment_analysis")
metrics

In [ ]:
models_prauc = sorted(list(map(lambda item: (item[0], item[1]["PR AUC"]), metrics.items())), key = lambda x: x[1])
print("Best model is {} with PR-AUC {}".format(models_prauc[-1][0], models_prauc[-1][1]))

## Topic Modelling

In [ ]:
fig = file_util.get_topics_html("LDA")
fig.update_layout(width = 700, height = 800)

In [ ]:
fig = file_util.get_topics_html("BERTopic")
fig.update_layout(width = 700, height = 1000)

In [ ]:
fig = file_util.get_topics_html("NMF")
fig.update_layout(width = 700, height = 550)

# Predict reviews_test.csv

In [ ]:
from src.models.predict import predict_sentiment_topic

In [ ]:
file_util.TEST_FILE_NAME

In [ ]:
file_util.best_sentiment_analysis_model

In [ ]:
test_bert = predict_sentiment_topic()
test_bert.head()

In [ ]:
test_output = test_bert.drop(["cleaned_text", "subtopic", "topic"], axis = 1)
test_output = test_output.rename(columns = {"partially_cleaned_text": "Text", "date": "Time", "sentiment": "predicted_sentiment", 
                            "sentiment_prob": "predicted_sentiment_prob"})

In [ ]:
test_output.head()

In [ ]:
test_output.to_csv("final_presentation/reviews_test_predictions_h2o2.ai.csv")

## Extra (Prediction with Other Models)

In [6]:
import yaml

with open("config.yml") as f:
     config_params = yaml.safe_load(f)

config_params["best_sentiment_analysis_model"] = "Logistic Regression"

with open("config.yml", "w") as f:
    yaml.dump(config_params, f)

In [ ]:
file_util.best_sentiment_analysis_model

In [ ]:
test_logreg = predict_sentiment_topic()
test_logreg.head()

In [ ]:
with open("config.yml") as f:
     config_params = yaml.safe_load(f)

config_params["best_sentiment_analysis_model"] = "LSTM"

with open("config.yml", "w") as f:
    yaml.dump(config_params, f)

In [ ]:
file_util.best_sentiment_analysis_model

In [ ]:
test_lstm = predict_sentiment_topic()
test_lstm.head()

In [ ]:
with open("config.yml") as f:
     config_params = yaml.safe_load(f)

config_params["best_sentiment_analysis_model"] = "BERT"

with open("config.yml", "w") as f:
    yaml.dump(config_params, f)

## Visualizations

In [ ]:
import pandas as pd
from src.visualisation.dashboard_viz import *

vis_df = reformat_data(test_bert)
pio.renderers.default = "svg"

### Visualizations for sentiments

In [ ]:
sentiment_pie_chart_fig = sentiment_pie_chart(vis_df)
sentiment_trend_fig = sentiment_line_chart_over_time(vis_df)
topics_sentiment_fig = topics_bar_chart(vis_df)

display(sentiment_pie_chart_fig.update_layout(width = 500, height = 300, title='Overall Sentiment Breakdown'))
display(sentiment_trend_fig.update_layout(title='Sentiment trend'))
display(topics_sentiment_fig.update_layout(title='Topics by Sentiment'))

### Visualizations for topics

In [ ]:
topics_pie_chart_fig = topics_pie_chart(vis_df)
topics_bar_chart_fig = topics_bar_chart_over_time(vis_df, time_frame='Q')
top_key_words_fig = visualise_all_topics(vis_df)

display(topics_pie_chart_fig.update_layout(width = 500, height = 300, title='Frequency of topics'))
display(topics_bar_chart_fig.update_layout(title='Topics over Time'))
display(top_key_words_fig)

### Visualizations for specific topic

In [ ]:
# Subtopics in each topic
select_topic = 'Drinks'

subtopic_fig = get_subtopics(vis_df, topic=select_topic)
subtopic_sentiment_fig = sentiment_pie_chart(vis_df[vis_df["topic"]==select_topic])

display(subtopic_sentiment_fig.update_layout(width = 500, height = 300,  title=f'Sentiment Breakdown for {select_topic}'))
display(subtopic_fig.update_layout(width = 500, height = 300))

# Unit Testing

In [ ]:
import src.unittest.unit_testing
from src.unittest.unit_testing import unit_test

In [ ]:
print("Methods in unit testing:", [method for method in dir(src.unittest.unit_testing) if method[:4] == "test"])

In [ ]:
unit_test()

# Modular Code

In [ ]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        if os.path.basename(root) == "__pycache__":
            continue
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

In [ ]:
list_files("src")

# OOP

In [ ]:
from src.utils.file_util import FileUtil
print("Methods in FileUtil:", [func for func in dir(FileUtil) if callable(getattr(FileUtil, func)) and not func.startswith("__")])
print("Attributes in FileUtil:", list(FileUtil().__dict__.keys()))

In [ ]:
from src.models.sentiment_analysis.train.bert import BERT
print("Methods in BERT:", [func for func in dir(BERT) if callable(getattr(BERT, func)) and not func.startswith("__")])
print("Attributes in BERT:", list(BERT().__dict__.keys()))

In [ ]:
from src.models.topic_modelling.train.lda import LDA
print("Methods in LDA:", [func for func in dir(LDA) if callable(getattr(LDA, func)) and not func.startswith("__")])
print("Attributes in LDA:", list(LDA().__dict__.keys()))

In [ ]:
from src.models.topic_modelling.train.bertopic import BERTopic_Module
print("Methods in BERTopic_Module:", [func for func in dir(BERTopic_Module) if callable(getattr(BERTopic_Module, func)) and not func.startswith("__")])
print("Attributes in BERTopic_Module:", list(BERTopic_Module().__dict__.keys()))

# Docstrings Examples

In [ ]:
help(FileUtil.put_csv)

In [ ]:
print(predict_sentiment_topic.__doc__)

In [ ]:
help(sentiment_analysis_train)

In [ ]:
help(topic_modelling_train)